In [ ]:
import json
import requests

In [ ]:
github_articles = []
for offset in range(0, 100000, 1000):
    query = f"https://p55oroem7k.execute-api.eu-west-1.amazonaws.com/prod/articles/?skip={offset}&limit=1000"
    res = requests.get(query).json()
    print(query + " -- " + str(len(res)))
    for a in res:
        if "github" in a["summary"].lower():
            github_articles.append(a)

In [ ]:
len(github_articles)

In [ ]:
with open("github_articles.json", "w") as fout:
    json.dump(github_articles, fout)

In [ ]:
with open("github_articles.json", "r") as fin:
    github_articles = json.load(fin)

In [ ]:
len(github_articles)

## Async IO version

In [ ]:
import asyncio
import httpx

async def request_one(offset, client):
    url = f"https://p55oroem7k.execute-api.eu-west-1.amazonaws.com/prod/articles/?skip={offset}&limit=1000"
    resp = await client.get(url, timeout=30)
    resp = resp.json()
    gh_articles = []
    try:
        for a in resp:
            if "github" in a["summary"].lower():
                gh_articles.append(a)
        return gh_articles
    except TypeError as e:
        print(resp)
        raise(e)

async def perform_requests():
    async with httpx.AsyncClient() as client:
        tasks = []
        for offset in range(0, 98000, 1000):
            tasks.append(
                request_one(offset=offset, client=client)
            )
        res = await asyncio.gather(*tasks)
    return res

In [ ]:
results = await perform_requests()
gh_articles = [a for r in results for a in r]